# 微分可能BP復号法

Copyright (c) 2022 Tadashi Wadayama  
Released under the MIT license  
https://opensource.org/licenses/mit-license.php

In [139]:
using LinearAlgebra
using Plots
gr()
using Random
Random.seed!(3)
using Flux

### 検査行列定義ファイル(alist形式)

このコードで利用する検査行列定義ファイルBCH_31_16_3_strip.alistは
下記のサイトからダウンロードして利用してください。

https://www.uni-kl.de/channel-codes/ml-simulation-results/

In [140]:
filename = "BCH_31_16_3_strip.alist"

"BCH_31_16_3_strip.alist"

### グローバル変数の設定

In [141]:
noise_std = 0.5
K = 100
bp_itr = 5

5

### Alist ファイルの読み込み

In [142]:
fp = open(filename, "r") 
dict = Dict()
rdict = Dict()

line = readline(fp)
u = split(line, " ") 
n = parse(Int64,u[1])
m = parse(Int64,u[2])
println("n = ", n) 
println("m = ", m)
line = readline(fp)
u = split(line, " ") 
cmax = parse(Int64,u[1])
rmax = parse(Int64,u[2]) 
println("cmax = ", cmax) 
println("rmax = ", rmax)
line = readline(fp)
line = readline(fp)
id = 1
for i in 1:n
    line = readline(fp) 
    u = split(line, " ") 
    for j in 1:cmax
        v = parse(Int64, u[j])
        if v != 0
            dict[(v,i)] = id
            rdict[id] = (v,i)
            id += 1         
        end
    end
end
esize = length(dict)

H = zeros(Int, m,n)
for x in keys(dict)
    H[x[1], x[2]] = 1
end

U = zeros(Int, n, esize)
for edge in dict
    j = edge[2]
    i = edge[1][2]
    U[i,j]=1
end

V = zeros(Int, m, esize)
for edge in dict
    j = edge[2]
    i = edge[1][1]
    V[i,j]=1
end

n = 31
m = 15
cmax = 7
rmax = 8


### パリティ検査関数

In [143]:
bmod(x) = x-2*floor(x/2)
function parity_check(x)
    if sum(bmod.(H*x)) > 0
        return 1
    else
        return 0
    end
end

parity_check (generic function with 1 method)

### 符号化関数　

注意: ただし、左下三角がゼロのalist ファイルに限る！！
左下三角がゼロでないalistファイルの場合にはencoding error が出る

In [144]:
function random_codeword()
    x = rand(0:1, n)
    for i in 1:m
        k = m - i + 1
        parity = 0
        for j in k+1:n
            parity += H[k,j]*x[j]
        end
        x[k] = bmod(parity)
    end
    p = parity_check(x)
    if p == 0
        return x
    else 
        println("encoding error!")
    end
    return x
end

random_codeword (generic function with 1 method)

In [145]:
bmod(7)

1.0

In [146]:
mod(7,2)

1

In [147]:
x = random_codeword()
parity_check(x)

0

### ミニバッチ生成関数

AWGN通信路, BPSK変調を仮定

In [148]:
function mini_batch(K)
    x = zeros(n,K)
    y = zeros(n,K)
    for j in 1:K
        x[:,j] = 1 .- 2*random_codeword()
        y[:,j] = x[:,j] + noise_std*randn(n)
    end
    return x,y
end

mini_batch (generic function with 1 method)

### 微分可能ビリーフプロパゲーション復号関数

In [149]:
eps = 1e-8
    
function BP_decoding(y, max_itr, xi)
    alpha = zeros(esize,K) 
    beta = zeros(esize,K)
    lambda = 2*y./noise_std^2
    for i in 1:max_itr
        beta = (U'*U-I)*alpha + U'*lambda
        beta = xi[:,i] .* beta    
        tmp = exp.((V'*V-I)*log.(abs.(tanh.(beta/2))))
        alpha_abs = 2*atanh.((1-eps)*hardtanh.(tmp))
        tmp = 1 .- 2*V'*bmod.(V*((-sign.(beta).+ 1)/2))
        alpha_sign = tmp .* sign.(beta)
        alpha = alpha_sign .* alpha_abs 
    end
    gamma = U*alpha + lambda
    return tanh.(gamma)
end

BP_decoding (generic function with 1 method)

### BP復号のデモ

In [150]:
x, y = mini_batch(K)
xi = ones(esize, bp_itr)
x̂ = sign.(BP_decoding(y, bp_itr, xi))
x̂ .!= x

31×100 BitMatrix:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  

### 学習プロセス

In [151]:
opt = Flux.ADAM(1e-2) 
train_itr = 1000
loss(x,y) = norm(x-y)^2

xi_train = ones(esize, bp_itr)
ps = Flux.params(xi_train)

for i in 1:train_itr
    x,y = mini_batch(K)
    gs = Flux.gradient(ps) do
        x̂ = BP_decoding(y, bp_itr, xi_train)
        loss(x, x̂)
    end
    Flux.Optimise.update!(opt, ps, gs)
    println(i)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [152]:
xi_train

120×5 Matrix{Float64}:
 0.680561  1.07233   0.808355  0.842541  0.551961
 0.858792  1.12562   1.08882   0.870019  1.22128
 0.738686  0.959727  1.1013    0.892392  1.02499
 1.04707   0.999215  1.1257    1.05581   0.789494
 0.812502  0.9081    0.705376  0.673044  0.847648
 0.547112  0.579457  0.892566  1.17789   0.88614
 0.981838  0.722922  0.710207  0.980413  0.525939
 0.882362  1.05867   0.899245  1.11395   0.676345
 0.85369   0.848373  0.654395  0.188433  1.0186
 1.29297   0.989019  0.475429  0.751001  0.546337
 0.591104  0.853366  1.33328   0.654046  0.52492
 1.13013   1.21482   1.04016   0.806212  1.15314
 0.721232  0.756664  0.838092  0.475009  0.700075
 ⋮                                       
 1.33534   0.600953  1.00013   0.796868  0.702937
 0.920712  0.781355  1.12608   0.782221  1.02104
 1.48373   0.817105  0.880358  0.465858  0.643721
 1.29467   0.618438  1.22499   0.811291  0.847872
 0.801032  1.06888   0.960319  0.826582  0.730672
 1.08301   0.496215  1.00698   0.571839  0.

### 微分可能ビリーフプロパゲーション復号法のビット誤り率

In [ ]:
total_syms = 0
error_syms = 0
num_loops = 1000

for i in 1:num_loops
    x, y = mini_batch(K)
    x̂ = BP_decoding(y, bp_itr, xi_train)
    total_syms += n*K
    error_syms += sum(sign.(x̂) .!= x)
end
println("total_syms = ", total_syms)
println("error_syms = ", error_syms)
println("symbols error rate = ", error_syms/total_syms)

### オリジナルのビリーフプロパゲーション復号法のビット誤り率

In [ ]:
total_syms = 0
error_syms = 0
num_loops = 1000
xi = ones(esize, bp_itr)

for i in 1:num_loops
    x, y = mini_batch(K)
    x̂ = BP_decoding(y, bp_itr, xi)
    total_syms += n*K
    error_syms += sum(sign.(x̂) .!= x)
end
println("total_syms = ", total_syms)
println("error_syms = ", error_syms)
println("symbols error rate = ", error_syms/total_syms)